In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import random
from torchvision.models import resnet50, ResNet50_Weights


In [2]:
class SiameseNetworkDataset(Dataset):
    def __init__(self, imageFolderDataset, transform=None):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
    
    def __getitem__(self, index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        should_get_same_class = random.randint(0, 1)
        if should_get_same_class:
            while True:
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] == img1_tuple[1]:
                    break
        else:
            while True:
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] != img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
        img0 = img0.convert("RGB")
        img1 = img1.convert("RGB")

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)

        return img0, img1, torch.from_numpy(np.array([int(img0_tuple[1] != img1_tuple[1])], dtype=np.float32))

    def __len__(self):
        return len(self.imageFolderDataset.imgs)


In [16]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        weights = ResNet50_Weights.DEFAULT
        self.backbone = resnet50(weights=weights)
        self.backbone.fc = nn.Identity()  # Remove the last fully connected layer
        self.fc1 = nn.Sequential(
            nn.Linear(2048, 500),  # Change to 2048 for resnet50
            nn.ReLU(inplace=True),
            nn.Linear(500, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 5)
        )
        self.softmax = nn.Softmax(dim=1)

    def forward_once(self, x):
        output = self.backbone(x)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

In [4]:
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = nn.functional.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

In [ ]:
import os
from PIL import Image
import numpy as np
import random
import os
import matplotlib.pyplot as plt

def train_siamese_network():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    dataset = datasets.ImageFolder(root='siamese-datasets/datasets/train', transform=transform)
    siamese_dataset = SiameseNetworkDataset(imageFolderDataset=dataset, transform=transform)
    train_loader = DataLoader(siamese_dataset, shuffle=True, num_workers=4, batch_size=4)

    model = SiameseNetwork().cuda()
    criterion = ContrastiveLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.00001)

    num_epochs = 200
    train_loss = []
    train_accuracy = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for i, data in enumerate(train_loader, 0):
            img0, img1, label = data
            img0, img1, label = img0.cuda(), img1.cuda(), label.cuda()
            optimizer.zero_grad()
            output1, output2 = model(img0, img1)
            loss_contrastive = criterion(output1, output2, label)
            loss_contrastive.backward()
            optimizer.step()

            running_loss += loss_contrastive.item()
            euclidean_distance = nn.functional.pairwise_distance(output1, output2)
            predicted = (euclidean_distance > 1.0).float()
            total += label.size(0)
            correct += (predicted == label).sum().item()

            if i % 10 == 0:
                print(f"Epoch {epoch+1}/{num_epochs}, Iter {i}, Loss: {loss_contrastive.item()}")

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total

        train_loss.append(epoch_loss)
        train_accuracy.append(epoch_accuracy)

        # Save the model checkpoint
        if not os.path.exists('checkpoints'):
            os.makedirs('checkpoints')
        torch.save(model.state_dict(), f'checkpoints/siamese_epoch_{epoch+1}.pth')

    # Save the final model
    torch.save(model.state_dict(), 'siamese_network.pth')

    # Plot loss and accuracy
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(range(1, num_epochs + 1), train_loss, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Over Epochs')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, num_epochs + 1), train_accuracy, label='Training Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training Accuracy Over Epochs')
    plt.legend()

    plt.tight_layout()
    plt.savefig('training_plot.png')
    plt.show()

if __name__ == "__main__":
    train_siamese_network()

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

def test_siamese_network(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            img0, img1, label = data
            img0, img1, label = img0.cuda(), img1.cuda(), label.cuda()
            output1, output2 = model(img0, img1)
            euclidean_distance = nn.functional.pairwise_distance(output1, output2)
            predicted = (euclidean_distance > 1.0).float()
            total += label.size(0)
            correct += (predicted == label).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    return accuracy

if __name__ == "__main__":
    # Chuẩn bị phép biến đổi và dữ liệu kiểm tra
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    test_dataset = datasets.ImageFolder(root='siamese-datasets/datasets/test', transform=transform)
    siamese_test_dataset = SiameseNetworkDataset(imageFolderDataset=test_dataset, transform=transform)
    test_loader = DataLoader(siamese_test_dataset, shuffle=False, num_workers=8, batch_size=2)

    # Tải mô hình đã huấn luyện
    model = SiameseNetwork().cuda()
    model.load_state_dict(torch.load('siamese_network.pth'))

    # Kiểm tra mô hình
    test_siamese_network(model, test_loader)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from torchvision.models import resnet50

# Hàm tải hình ảnh và chuyển đổi sang dạng Tensor
def load_image(image_path):
    image = Image.open(image_path)
    
    # Nếu hình ảnh có 4 channels (RGBA), chuyển đổi thành 3 channels (RGB)
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    
    # Chuyển đổi hình ảnh thành dạng Tensor và chuẩn hóa
    loader = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = loader(image).unsqueeze(0)
    return image

# Tải mô hình đã được huấn luyện
model = SiameseNetwork()
model.load_state_dict(torch.load("siamese_network.pth", map_location=torch.device('cpu')))
model.eval()

# Hàm để đánh giá sự tương đồng giữa hai hình ảnh
def evaluate_similarity(image_path1, image_path2):
    image1 = load_image(image_path1)
    image2 = load_image(image_path2)
    
    with torch.no_grad():
        output = model(image1, image2)
    
    # Lấy phần tử đầu tiên từ tuple (giả sử chỉ có một tensor trong tuple)
    tensor_output = output[0]
    
    # Kiểm tra kích thước của tensor_output
    if isinstance(tensor_output, torch.Tensor) and tensor_output.numel() == 1:
        similarity_score = tensor_output.item()
    else:
        # Xử lý trường hợp kích thước tensor_output không phải là 1
        # Ví dụ: tính trung bình của các phần tử trong tensor_output
        similarity_score = torch.mean(tensor_output).item()
    
    return similarity_score


# Ví dụ sử dụng hàm đánh giá
image1_path = "siamese-datasets/datasets/test/normal/normal (10).png"
image2_path = "siamese-datasets/datasets/test/unnormal/unnormal (27).png"
similarity_score = evaluate_similarity(image1_path, image2_path)
print("Similarity Score:", similarity_score)
